#### Importar Librerias

In [1]:
import ray
from ray import serve
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError

#### Conecta a un clúster de Ray ya iniciado

In [2]:
try:
    ray.init(address="ray://172.26.56.134:10001")
    serve.start(detached=True)
    print("Conexion correcta al servicio Ray ...")
except Exception as ex:
    print(f"Ray: Se produjo el siguiente error : {ex} ")

Conexion correcta al servicio Ray ...


#### Configuracion del Modelo

In [3]:
try:
    app = FastAPI(
        title = "Import Predictor API",
        description = "Import Pipeline online inference"
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code = 422,
            content = { "cod": 422, "msg": "; ".join(mensajes), "data": None, "errors": exc.errors(), },
        )
    
    class Item(BaseModel):
        num_empr_trans:     str
        cod_lugar_descarga: str
        tipdocempdoctra:    str
        cod_enti_matricula: str
        cod_tip_docu_c0008: str
        cnt_pes_brut_c0008: float
        cnt_bulto_c0008:    float
        cnt_volumen_c0008:  float
        cnt_container:      float
        cod_puer_des_c0008: str
        cod_natucarga:      str
        tpoemp_partic_cn:   str
        tamano_partic_cn:   str
        identi_partic_cn:   str
        rat_pes_bult_c0008: float
        segmento_partic_cn: str
    
    @serve.deployment(
        route_prefix="/v1/gestionriesgo/evaluacionriesgoaduanero/contrab-carga-iamc/score/e/dtscore",
        ray_actor_options={"num_cpus": 0.1, "num_gpus": 0},
        name = "contrab-carga-iamc"
    )
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self):
            mlflow.set_tracking_uri("http://172.26.56.133:5000")
            model_uri = "models:/MLOPS_Modelo_IAMC_Carga/Production"
            try:
                self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: Item):
            try:
                #payload = await request.json()
                payload = item.dict()
                df = pd.DataFrame([payload])
                score = float(self.model.predict_proba(df)[0][1])
                return {"cod": 200, "msg": "Score calculated successfully.", "data": {"em_eventprobability": score}, "errors": None }
            except Exception as e:
                #logger.error(f"Error in import prediction: {e}")
                return {"cod": 500, "msg": "Error calculating score", "data": None, "errors": str(e)}
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

#### Desplegar el Modelo

In [4]:
try:
    ModelMlFlow.deploy()
    print("Modelo Desplegado")
except Exception as ex:
    serve.get_deployment("contrab-carga-iamc").delete()
    print(f"Ray: Se produjo el siguiente error : {ex} ")

Modelo Desplegado


http://172.26.56.134:8000/v1/gestionriesgo/evaluacionriesgoaduanero/contrab-carga-iamc/score/e/dtscore/docs

In [5]:
serve.list_deployments()

{'MMD983402025000001': Deployment(name=MMD983402025000001,version=None,route_prefix=/v1/gestionriesgo/evaluacionriesgoaduanero/sobrevalu-expo-nac/score/e/damscore),
 'contrab-carga-iamc': Deployment(name=contrab-carga-iamc,version=None,route_prefix=/v1/gestionriesgo/evaluacionriesgoaduanero/contrab-carga-iamc/score/e/dtscore)}

In [6]:
#serve.get_deployment("contrab-carga-iamc").delete()